In [216]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
# import sage
# from scipy.stats import rankdata
from scipy.stats import ttest_ind, t
import sys
from helper import *
from shapley_sampling1 import *
# from os.path import join
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, WeightedRandomSampler
import warnings
warnings.filterwarnings('ignore')

### Load dataset

In [232]:
# np.random.seed(1)
# data = pd.read_csv("Data/brca_small.csv")
# X = data.values[:, :-1][:,:20]
# Y = data.values[:, -1]
# Y = (Y==2).astype(int) # Formulate as binary classification problem

# # Split data
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=100, random_state=0)

# # Normalize
# mean = X_train.mean(axis=0)
# std = X_train.std(axis=0)
# X_train = (X_train - mean) / std
# X_test = (X_test - mean) / std
# d = X_train.shape[1]
# mapping_dict = None

# # Compute mean and covariance of training data
# feature_means = np.mean(X_train, axis=0)
# cov_mat = correct_cov(np.cov(X_train, rowvar=False))

In [244]:
# df = sage.datasets.credit()
# df.to_csv("Data/credit.csv")
df = pd.read_csv("Data/credit.csv", index_col=0)

# Property, other installment, housing, job, status of checking act, credit history, purpose, savings, employment since, marital status, old debtors
n = df.shape[0]
X_df = df.drop(["Good Customer"], axis=1)
y = df["Good Customer"]

categorical_columns = [
    'Checking Status', 'Credit History', 'Purpose', #'Credit Amount', # It's listed but has 923 unique values
    'Savings Account/Bonds', 'Employment Since', 'Personal Status',
    'Debtors/Guarantors', 'Property Type', 'Other Installment Plans',
    'Housing Ownership', 'Job', #'Telephone', 'Foreign Worker' # These are just binary
]
X_binarized = pd.get_dummies(X_df, columns=categorical_columns)

mapping_dict = {}
for i, col in enumerate(X_df.columns):
    bin_cols = []
    for j, bin_col in enumerate(X_binarized.columns):
        if bin_col.startswith(col):
            bin_cols.append(j)
    mapping_dict[i] = bin_cols

np.random.seed(1)
X_norm = (X_binarized-X_binarized.min())/(X_binarized.max()-X_binarized.min())
n_train = round(n*0.8)
train_idx = np.random.choice(n, n_train, replace=False)
X_train, y_train = X_norm.iloc[train_idx].to_numpy(), y.iloc[train_idx].to_numpy()
test_idx = np.setdiff1d(np.arange(n),train_idx)
X_test, y_test = X_norm.iloc[test_idx].to_numpy(), y.iloc[test_idx].to_numpy()
d = X_df.shape[1]
d_bin = X_train.shape[1]

In [245]:
# Bank dataset
# import sage
# from sklearn.model_selection import train_test_split

# df = sage.datasets.bank()
# df.Default = df.Default.map({'no': 0, 'yes': 1})
# df.Housing = df.Housing.map({'no': 0, 'yes': 1})
# df.Loan = df.Loan.map({'no': 0, 'yes': 1})
# df.columns = df.columns.str.replace(' ', '_')
# df_bin = pd.get_dummies(df)

# # That's literally it. Note this mapping dict is a little different.
# train, test = train_test_split(
#     df_bin.values, test_size=int(0.1 * len(df_bin.values)), random_state=123)

# suc_idx = df_bin.columns.get_loc("Success")
# y_train = train[:, suc_idx].copy().astype(int)
# y_test = test[:, suc_idx].copy().astype(int)
# X_train = np.delete(train, suc_idx, axis=1).astype("float64")
# X_test = np.delete(test, suc_idx, axis=1).astype("float64")

# # Get mapping dict
# X_df = df_bin.drop(["Success"], axis=1)
# mapping_dict = {}
# for i, col in enumerate(df.columns):
#     bin_cols = []
#     for j, bin_col in enumerate(X_df.columns):
#         if bin_col.startswith(col):
#             bin_cols.append(j)
#     mapping_dict[i] = bin_cols

In [246]:
# # Adult census income dataset
# import shap
# X, y = shap.datasets.adult()
# X_display, y_display = shap.datasets.adult(display=True)
# X_binarized = pd.get_dummies(X_display)

# mapping_dict = {}
# for i, col in enumerate(X_display.columns):
#     bin_cols = []
#     for j, bin_col in enumerate(X_binarized.columns):
#         if bin_col.startswith(col):
#             bin_cols.append(j)
#     mapping_dict[i] = bin_cols

# X_norm = (X_binarized-X_binarized.min())/(X_binarized.max()-X_binarized.min())
# y_int = y_display.astype("int8")

# # Split into training and test sets
# np.random.seed(1)
# n, d = X_norm.shape
# n_train = round(n*0.75)
# train_idx = np.random.choice(n, size=n_train, replace=False)
# X_train_pd, y_train = X_norm.iloc[train_idx], y_int[train_idx]
# X_train = X_train_pd.to_numpy()

# test_idx = np.setdiff1d(list(range(n)), train_idx)
# X_test_pd, y_test = X_norm.iloc[test_idx], y_int[test_idx]
# X_test = X_test_pd.to_numpy()


### Define neural network, train, and move to numpy
- Trains with balanced weights

In [247]:
class TwoLayerNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(TwoLayerNet, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.tanh = nn.Tanh()
        self.linear2 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        out = self.linear1(x)
        out = self.tanh(out)
        out = self.linear2(out)
        out = self.softmax(out)
        return out

# Convert the input and label data to PyTorch tensors
inputs = torch.tensor(X_train, dtype=torch.float32)
labels = torch.tensor(y_train, dtype=torch.long)

# Compute the class weights
class_counts = torch.bincount(labels)
num_samples = len(labels)
class_weights = 1.0 / class_counts.float()
sample_weights = class_weights[labels]

# Create a sampler with balanced weights
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=num_samples, replacement=True)

# Create a DataLoader with the sampler
dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset, batch_size=32, sampler=sampler)
# dataloader = DataLoader(dataset, batch_size=32)

torch.manual_seed(0)

# Create an instance
net = TwoLayerNet(input_size=X_train.shape[1], hidden_size=50, output_size=2)
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)#.01

# Iterate over the training data in batches
num_epochs = 20

# Train the network for the specified number of epochs
for epoch in range(num_epochs):
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    # print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

def neural_net(x):
    output = net(x)[0,1] if x.shape[0]==1 else net(x)[:,1]
    return output

def compute_hessian(x):
    if not torch.is_tensor(x):
        x = torch.tensor(x, dtype=torch.float32)
    dim = x.shape[1]
    hessian = torch.autograd.functional.hessian(neural_net, x)
    hessian = hessian.reshape((dim,dim)).detach().numpy()
    return hessian

def model(x):
    if not torch.is_tensor(x):
        x = torch.tensor(x, dtype=torch.float32)
    return neural_net(x).detach().numpy()

print("Class imbalance: {}%".format(round(100*(max(np.mean(y_test), 1-np.mean(y_test))))))
Y_preds = (model(X_test) > 0.5).astype("int")
print("NN {}% accuracy".format(round(np.mean(Y_preds == y_test)*100)))



Class imbalance: 72%
NN 68% accuracy


In [248]:
%run helper
# Compute mean and covariance of training data
feature_means = np.mean(X_train, axis=0)
cov_mat = correct_cov(np.cov(X_train, rowvar=False))

# Select point and compute its gradient and hessian
xloc = X_test[0:1]
xloc_torch = torch.tensor(xloc, dtype=torch.float32).requires_grad_(True)
y_pred = net(xloc_torch)[0,1]
y_pred.backward()
gradient = xloc_torch.grad.detach().numpy().reshape((xloc.shape[1], 1))
hessian = compute_hessian(xloc)

# Obtain true SHAP values and verify their feasibility
true_shap_vals = compute_shap_vals_quadratic(xloc, gradient, hessian, feature_means, cov_mat, mapping_dict=mapping_dict)

y_pred = y_pred.detach().numpy()
def approx(input):
    return f_second_order_approx(y_pred,input,xloc,gradient,hessian)

## Shapley Sampling

### Algo 1a: Non-adaptive. Keep running for all features until convergence

In [170]:
%run shapley_sampling1
# np.random.seed(1)
K = 3
shap_vals, diffs_all_feats = shapley_sampling_basic(approx, X_train, xloc, K, 
                                                    n_perms_btwn_tests=10, n_init=50, n_samples_per_perm=2,
                                                    alpha=0.2, mapping_dict=mapping_dict)
print(diffs_all_feats.shape)


KeyboardInterrupt: 

### Algo 1b: Adaptive. Keep running on features that fail pairwise test

Number of times computing $v_x(S \cup j) - v_x(S)$, with neural net

In [ ]:
# np.random.seed(1)
shap_vals2, diffs_all_feats2 = shapley_sampling_adaptive(approx, X_train, xloc, K, 
                                        mapping_dict=mapping_dict, alpha=0.2, 
                                        n_perms_btwn_tests=10, n_init=50, n_samples_per_perm=2,
                                        K_thru_rest=False)
print(sum([len(diffs_all_feats2[j]) for j in range(len(diffs_all_feats2))]))
print(np.product(diffs_all_feats.shape))

print(get_ordering(true_shap_vals))
print(get_ordering(shap_vals))
print(get_ordering(shap_vals2))

50520
5400
[ 2 11  0  8  1 16  7 13  6  3  4 19 18 17  5 12 10  9 15 14]
[ 2 11  8  1  0 16  6  7  4  5 13 19  3 18 17 12 10  9 14 15]
[ 2 11  0  8 16  1 13  7  3  5  4 19  6 17 18 12  9 14 10 15]


In [ ]:
print(model(xloc) - np.mean(model(X_train)))
print(np.sum(true_shap_vals))
print(np.sum(shap_vals))
print(np.sum(shap_vals2))

0.3060394
0.9601558484137058
1.0561681243313785
0.9215477463980719


## Speed-up on model SHAP values

Census
- K=1: No change
- K=2: 2x
- K=3, 5: 2.5x
- K=7: 3.5x

BRCA (slow)
- K=1: 5x
- K=2: ~10x (?)

Credit
- K=1: 20x
- K=2: 10x
- K=3: 20x

In [229]:
cts_nonadaptive, cts_adaptive = [], []
K = 2
for i in range(20):
    # print(i)
    shap_vals, diffs_all_feats = shapley_sampling_basic(model, X_train, xloc, K, 
                                                    n_perms_btwn_tests=10, n_init=50, n_samples_per_perm=5,
                                                    alpha=0.2, mapping_dict=mapping_dict)
    shap_vals2, diffs_all_feats2 = shapley_sampling_adaptive(model, X_train, xloc, K, 
                                        mapping_dict=mapping_dict, alpha=0.2, 
                                        n_perms_btwn_tests=10, n_init=30, n_samples_per_perm=5)
    cts_nonadaptive.append(np.product(diffs_all_feats.shape))
    cts_adaptive.append(sum([len(diffs_all_feats2[j]) for j in range(len(diffs_all_feats2))]))
    print(np.round(np.mean(cts_nonadaptive)/np.mean(cts_adaptive), 2))
    

2.33


KeyboardInterrupt: 

In [74]:
print(np.round(np.mean(cts_nonadaptive)/np.mean(cts_adaptive), 2))

1.85


## Simulation
### Quadratic Approximation
Not phenomenal. 
BRCA
- K=1: 13%
- K=2: 24%

Census
- K=1: 34%
- K=2: 33%

Credit
- K=2: 38%
- K=5: 32%

In [249]:
%run shapley_sampling1
# np.random.seed(2)
K = 5
true_order = np.argsort(np.abs(true_shap_vals))[::-1]
true_top_K = true_order[:K]
same_top_K = []
for i in range(100):
    # print(i)
    shap_vals2, diffs_all_feats2 = shapley_sampling_adaptive(approx, X_train, xloc, K, 
                                        mapping_dict=mapping_dict, alpha=0.2, 
                                        n_perms_btwn_tests=10, n_samples_per_perm=2,
                                        n_init=100)
    est_top_K = np.argsort(np.abs(shap_vals2))[::-1][:K]
    has_same_top_K = np.array_equal(true_top_K, est_top_K)
    same_top_K.append(has_same_top_K)
    if (i+1) % 5==0: print(i+1, np.round(1-np.mean(same_top_K),2)) # FWER
np.round(1-np.mean(same_top_K),2) 

5 0.6
10 0.4
15 0.47
20 0.4
25 0.4
30 0.47
35 0.46
40 0.45
45 0.44
50 0.4
55 0.38
60 0.37
65 0.38
70 0.37
75 0.37
80 0.4
85 0.39
90 0.38
95 0.38


KeyboardInterrupt: 

### Neural net itself
- This seems to controls FWER...

BRCA
- K=1, observe 22% FWER
- K=2, observe 30% FWER (slow)

Credit
- K=1, observe 35% FWER
    - Then ran for 1000 iters: only 24% FWER
- K=2, observe 29% FWER
- K=3, observe 24% FWER

Census
- K=1-5, observe ~0% FWER (3 and 4, 2%)
- K=6, observe 15% FWER
- K=7, observe 26% FWER

In [243]:
%run shapley_sampling1
# np.random.seed(2)
K = 2
top_K = []
for i in range(100):
    shap_vals2, diffs_all_feats2 = shapley_sampling_adaptive(model, X_train, xloc, K, 
                                        mapping_dict=mapping_dict, alpha=0.2, 
                                        n_perms_btwn_tests=10, n_samples_per_perm=1,
                                        n_init=100)
    est_top_K = np.argsort(np.abs(shap_vals2))[::-1][:K]
    top_K.append(est_top_K)
    if (i+1) % 5==0: 
        print(i+1)

5


KeyboardInterrupt: 

In [242]:
top_K = np.array(top_K)
most_common_row = mode_rows(top_K)
ct = 0
for i in range(top_K.shape[0]):
    if np.array_equal(most_common_row, top_K[i]):
        ct += 1
print(np.round(1 - ct / top_K.shape[0], 2)) # FWER
print([len(diffs_all_feats2[j]) for j in range(len(diffs_all_feats2))])


0.2
[50, 50, 50, 50, 5950, 9600, 320, 50, 4160, 240, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50]
